In [4]:
from yahoo_fin.stock_info import get_data
from datetime import date
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import panel as pn
from panel.interact import interact
from MCForecastTools import MCSimulation
pn.extension("plotly")
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Sebs1\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [5]:
portfolio_returns = pd.read_csv(r'data/portfolio_returns.csv').set_index('Date')
combined_portfolios = pd.read_csv(r'data/combined_portfolios.csv').set_index('Date')
combined_returns = pd.read_csv(r'data/combined_returns.csv').set_index('Date')
monte_carlo_df = combined_portfolios.dropna()
monte_carlo_dataframe = pd.read_csv(r'data/monte_carlo_dataframe.csv')
monte_carlo_dataframe = monte_carlo_dataframe.rename({'Unnamed: 0': 'Date'}, axis=1)
monte_carlo_dataframe = monte_carlo_dataframe.set_index('Date')
index_names = [
["BTC", "ETH", "SPY", "TLT", "GLD", "USD", "XLE", "XLF"],
["close", "close","close","close","close","close","close","close"]
# ["BTC", "ETH", "SPY", "TLT", "GLD", "USD"],
# ["close", "close","close","close","close","close"]
]
index_tuples = list(zip(*index_names))
new_index = pd.MultiIndex.from_tuples(index_tuples,names=["first", "second"])
monte_carlo_dataframe.columns = new_index
monte_carlo_dataframe



first,BTC,ETH,SPY,TLT,GLD,USD,XLE,XLF
second,close,close,close,close,close,close,close,close
Date,,,,,,,,
2016-01-04,433.091003,0.954480,201.020004,102.889999,121.449997,98.870003,60.299999,18.984566
2016-01-05,431.959991,0.950176,201.360001,103.180000,120.959999,99.400002,60.529999,19.057676
2016-01-06,429.105011,0.950860,198.820007,104.669998,122.589996,99.180000,58.200001,18.765232
2016-01-07,458.048004,0.942005,194.050003,106.150002,122.809998,98.220001,56.779999,18.237206
2016-01-08,453.230011,0.986789,191.919998,105.680000,123.360001,98.540001,56.049999,17.952885
2016-01-11,448.428009,1.061600,192.110001,104.739998,122.010002,98.730003,54.849998,17.977255
2016-01-12,435.690002,1.141000,193.660004,104.209999,123.769997,98.970001,54.980000,18.115353
2016-01-13,432.371002,1.125200,188.830002,104.720001,124.989998,98.930000,53.660000,17.644192


In [6]:
def get_port_returns_graph():
    port_returns_fig = px.line(portfolio_returns)
    port_returns_fig.update_layout(
    title='Volatility',
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True
    )
    return port_returns_fig

def get_vol_graph():
    cumulative_returns = (1 + portfolio_returns).cumprod()
    volatility_fig = px.line(cumulative_returns)
    volatility_fig.update_layout(
        title='Volatility',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return volatility_fig

def get_vol_box():
    vol_box_fig = go.Figure()
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['BTC'], name='Bitcoin',
                    marker_color = 'indianred', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['ETH'], name = 'Ethereum',
                    marker_color = 'lightseagreen', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['SPY'], name='S&P 500',
                    marker_color = 'blue', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['TLT'], name = 'Bonds (TLT)',
                    marker_color = 'grey', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['USD'], name = 'US Dollar',
                    marker_color = 'green', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['GLD'], name='Gold',
                    marker_color = 'gold', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['XLE'], name = 'Energy (XLE)',
                    marker_color = 'red', boxpoints=False))
    vol_box_fig.add_trace(go.Box(y=portfolio_returns['XLF'], name = 'Financials (XLF)',
                    marker_color = 'black', boxpoints=False))

    vol_box_fig.update_layout(
        title='Daily Return Distributions',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=False
    )

    return vol_box_fig

def get_rolling():
    # Calculate the rolling standard deviation for all portfolios using a 21-day window
    roll = portfolio_returns.rolling(window=120).std()
    # Plot the rolling standard deviation
    roll_120_fig = px.line(roll)
    roll_120_fig.update_layout(
        title='Rolling Returns',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return roll_120_fig

def get_ewm():
    ewm_calc = portfolio_returns.ewm(halflife=120).std()
    roll_120_line_fig = px.line(ewm_calc)
    roll_120_line_fig.update_layout(
        title='Rolling Returns',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return roll_120_line_fig

def get_sharpe_ratios():
    sharpe_ratios = (portfolio_returns.mean() * 252) / (portfolio_returns.std() * np.sqrt(252))
    sharpe_bar_fig = px.bar(sharpe_ratios,labels=dict(x="Assets", y="Sharpe Ratio", color="Sharpe Ratio"),color = sharpe_ratios, color_continuous_scale='Bluered_r')
    sharpe_bar_fig.update_layout(
    title='Sharpe Ratios',
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True
    )
    return sharpe_bar_fig

def get_correlation():
    price_correlation_new = portfolio_returns.corr()
    heatmap_fig = px.imshow(price_correlation_new, color_continuous_scale='Bluered_r', labels = dict(color='Corerelation'))
    heatmap_fig.update_layout(
        title='Individual Asset Correlations',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return heatmap_fig

def get_cumulative_portfolios():
    cum_returns = px.line(combined_portfolios)
    cum_returns.update_layout(
        title='5 Year Cumulative Portfolio Returns',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return cum_returns

def get_portfolio_sharpe_ratios():
    portfolio_sharpe_ratios = (combined_returns.mean() * 252) / (combined_returns.std() * np.sqrt(252))
    portfolio_sharpe_fig = px.bar(portfolio_sharpe_ratios,labels=dict(x="Assets", y="Sharpe Ratio", color="Sharpe Ratio"),color = portfolio_sharpe_ratios, color_continuous_scale='Bluered_r')
    portfolio_sharpe_fig.update_layout(
        title='Portoflio Sharpe Ratios',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return portfolio_sharpe_fig

def get_traditional_mc():
    mc_traditional = MCSimulation(
    monte_carlo_dataframe,
    weights = [0,0,0.50,0.40,0.05,0.05,0,0],
    num_simulation = 200,
    num_trading_days = 252*5)
    mc_trad_px = mc_traditional.calc_cumulative_return()
    mc_trad_fig = px.line(mc_trad_px)
    mc_trad_fig.update_layout(
        title='Traditional Portfolio Monte Carlo Simulation',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return mc_trad_fig
    
def get_btc_mc():
    mc_btc_port = MCSimulation(
    monte_carlo_dataframe,
    weights = [0.10,0,0.50,0.40,0,0,0,0],
    num_simulation = 200,
    num_trading_days = 252*5)
    mc_btc_px = mc_btc_port.calc_cumulative_return()
    mc_btc_fig = px.line(mc_btc_px)
    mc_btc_fig.update_layout(
        title='Bitcoin Portfolio Monte Carlo Simulation',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return mc_btc_fig
    
    
def get_eth_mc():
    mc_eth_port = MCSimulation(
    monte_carlo_dataframe,
    weights = [0,0.10,0.50,0.40,0,0,0,0],
    num_simulation = 200,
    num_trading_days = 252*5)
    mc_eth_px = mc_eth_port.calc_cumulative_return()
    mc_eth_fig = px.line(mc_eth_px)
    mc_eth_fig.update_layout(
        title='Etherium Portfolio Monte Carlo Simulation',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return mc_eth_fig

    
def get_alpha_mc():
    mc_alpha_portfolio = MCSimulation(
    monte_carlo_dataframe,
    weights = [0.07,0.08,0.27,0.2,0.24,0.15,0,0],
    num_simulation = 200,
    num_trading_days = 252*5)
    mc_alpha_px = mc_alpha_portfolio.calc_cumulative_return()
    mc_alpha_fig = px.line(mc_alpha_px)
    mc_alpha_fig.update_layout(
        title='Alpha Portfolio Monte Carlo Simulation',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return mc_alpha_fig
    
    
def get_port_sharpes():
    portfolio_sharpe_ratios = (combined_returns.mean() * 252) / (combined_returns.std() * np.sqrt(252))
    portfolio_sharpe_fig = px.bar(portfolio_sharpe_ratios,labels=dict(x="Assets", y="Sharpe Ratio", color="Sharpe Ratio"),color = portfolio_sharpe_ratios, color_continuous_scale='Bluered_r')
    portfolio_sharpe_fig.update_layout(
        title='Portoflio Sharpe Ratios',
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )
    return portfolio_sharpe_fig
    
    
def get_histogram():
    histogram_fig = px.histogram(portfolio_returns)
    histogram_fig.update_layout(
    title='Histogram Showing Standard Deviation of Assets',
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True,
    barmode = 'stack'
    )
    return histogram_fig
    
    
def get_all_histograms():
    all_histo_fig = make_subplots(rows=4, cols=2)
    trace0 = go.Histogram(x=portfolio_returns['BTC'],name='BTC')
    trace1 = go.Histogram(x=portfolio_returns['ETH'],name='ETH')
    trace2 = go.Histogram(x=portfolio_returns['SPY'],name='SPY')
    trace3 = go.Histogram(x=portfolio_returns['TLT'],name='TLT')
    trace4 = go.Histogram(x=portfolio_returns['GLD'],name='GLD')
    trace5 = go.Histogram(x=portfolio_returns['TLT'],name='TLT')
    trace6 = go.Histogram(x=portfolio_returns['XLE'],name='XLE')
    trace7 = go.Histogram(x=portfolio_returns['XLF'],name='XLF')
    all_histo_fig.append_trace(trace0, 1, 1)
    all_histo_fig.append_trace(trace1, 1, 2)
    all_histo_fig.append_trace(trace2, 2, 1)
    all_histo_fig.append_trace(trace3, 2, 2)
    all_histo_fig.append_trace(trace4, 3, 1)
    all_histo_fig.append_trace(trace5, 3, 2)
    all_histo_fig.append_trace(trace6, 4, 1)
    all_histo_fig.append_trace(trace7, 4, 2)
    all_histo_fig.update_layout(
    title='Histogram Showing Standard Deviation of Assets',
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True)

    return all_histo_fig

In [7]:
overview_row = pn.Row(get_cumulative_portfolios(),get_port_sharpes())

In [8]:
trad_row = pn.Row(
        pn.Row('# Traditional Portfolio'),
        get_traditional_mc(),
        '##There is a 95% chance that an initial investment of $1000 in the portfolio over the next 5 years will end within in the range of $1046.48 and $2876.67')

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


In [9]:
btc_row = pn.Row(
    pn.Row('# Bitcoin Portfolio'),
    get_btc_mc(),
    '## There is a 95% chance that an initial investment of $1000 in the portfolio over the next 5 years will end within in the range of $1686.03 and $5516.51')

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


In [10]:
eth_row = pn.Row('# Etherium Portfolio',
    pn.Row(get_eth_mc(),
    '## There is a 95% chance that an initial investment of $1000 in the portfolio over the next 5 years will end within in the range of $2656.66 and $10589.95'))

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


In [11]:
alpha_row = pn.Row(
    pn.Row('# Alpha Portfolio'),
    pn.Row(get_alpha_mc(),
    '## There is a 95% chance that an initial investment of $1000 in the portfolio over the next 5 years will end within in the range of $2970.51 and $9815.35'))

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


In [12]:
overview_column = pn.Column(
    '# How Does Each Portfolio Stack Up?',
    pn.Row(get_cumulative_portfolios(),
    get_port_sharpes())
)

In [13]:
mc_column = pn.Column(
   pn.Row(trad_row,btc_row),pn.Row(eth_row,alpha_row)
)

In [14]:
vol_column = pn.Column(
    pn.Row('# At a Glance - Volatility of Assets',
    get_vol_box()), 
    pn.Row(get_vol_graph(),
    get_port_returns_graph())
)


In [15]:
roll_column = pn.Column(
    pn.Row('# At a Glance - Rolling 21 Day Average of Assets'),
    pn.Row(get_rolling(),
    get_ewm())
)


In [16]:
sharpe_column = pn.Column(
    pn.Row('# At a Glance - Sharpe Ratios of Each Asset'),
    pn.Row(get_all_histograms(),
    get_sharpe_ratios())
)


In [17]:
heat_column = pn.Column(
    '# Heatmap Correlations Related to Each Asset',
    get_correlation()
)


In [20]:
tabs = pn.Tabs(
    ("Volatility",vol_column),
    ("Rolling Averages",roll_column),
    ("Sharpe Ratios",sharpe_column),
    ("Heatmap",heat_column),
    ("Monte Carlo", mc_column),
    ("Conclusion", overview_column),
)

In [19]:
tabs.show()

Launching server at http://localhost:56634
